# EVID analysis

In [4]:
from src.model import EpiModel
import pandas as pd
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
from src.move_distribution import fillTensor
import numpy as np
import json
from os import path
# %matplotlib nbagg
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configs

In [5]:
experiment_name = 'all_maks'
config_path = path.join('configs',f'conf_{experiment_name}.json')
checkpoint_directory = path.join('experiments', experiment_name)
city_plan_path = 'final_filtered_yerevan_plan.csv'
start_date = datetime(2021, 4, 20, 0, 0)
population = 1080311
initial_infected = 100
plot_every = 1

Please add all additional parameters to `other_configs`

In [6]:
severity_dist={"asymptomatic": 0.24, "mild": 0.56, "severe": 0.2}
infection_countdown_dist={"loc": 48, "scale": 7}
mortality_rate={"asymptomatic": 0.000001 / 24, "mild": 0.001 / 24, "severe": 0.125 / 24}

In [7]:
other_configs = {'severity_dist': severity_dist,
                 'infection_countdown_dist': infection_countdown_dist,
                 'mortality_rate': mortality_rate,
                 'healing_period': 24*14,
                 'save_every': 1}

In [8]:
building_types = ['cafe', 'church', 'hospital', 'kindergarten', 'school', 'shop', 'sport', 'university', 'work',
                  'self.address']
age_dist = (0.0688, 0.196, 0.1869, 0.4431, 0.1052)
building_level_dist = ((1, 2, 3, 4, 5, 6, 7, 8, 9, 14),
      (0.00455, 0.03185, 0.04736, 0.1878, 0.25584, 0.02813, 0.00807, 0.09121, 0.23909, 0.1061))

## Initializing the model

In [6]:
with open(config_path) as json_file:
    configs = json.load(json_file)
city_building_plan = pd.read_csv(city_plan_path)

In [7]:
move_distribution_tensor = np.zeros(shape=(7, 5, 24, 10))
fillTensor(move_distribution_tensor)

In [8]:
model = EpiModel(population, configs['districts'], start_date, move_distribution_tensor, building_types, building_params=building_level_dist, 
                 age_dist=age_dist,
                 initial_infected=initial_infected,
                 data_frame=city_building_plan,
                 inf_prob_args=configs['compute_inf_prob_arguments'],
                 contact_prob=configs['contact_prob'],
                 checkpoint_directory=checkpoint_directory,
                 **other_configs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9700/9700 [00:00<00:00, 66738.54it/s]


In [9]:
n_step = 744
for i in range(0, n_step+1):
    date = model.date
    t = time.time()
    model.step()
    d = time.time() - t
    r = model.datacollector.get_model_vars_dataframe().iloc[-1].values
    print(f'{i}: {date} - Duration: {d},  I: {r[0]}, N: {r[1]}, D: {r[2]}, H: {r[3]}, P: {r[4]}')


0: 2021-04-20 00:00:00 - Duration: 25.96477246284485,  I: 100, N: 1080211, D: 0, H: 0, P: 5000
1: 2021-04-20 01:00:00 - Duration: 24.815633058547974,  I: 102, N: 1080209, D: 0, H: 0, P: 5000
2: 2021-04-20 02:00:00 - Duration: 23.65218424797058,  I: 102, N: 1080209, D: 0, H: 0, P: 5000
3: 2021-04-20 03:00:00 - Duration: 22.54088592529297,  I: 105, N: 1080206, D: 0, H: 0, P: 5000
4: 2021-04-20 04:00:00 - Duration: 22.35632586479187,  I: 106, N: 1080205, D: 0, H: 0, P: 5000
5: 2021-04-20 05:00:00 - Duration: 21.620571851730347,  I: 108, N: 1080203, D: 0, H: 0, P: 5000
6: 2021-04-20 06:00:00 - Duration: 21.688262224197388,  I: 109, N: 1080202, D: 0, H: 0, P: 5000
7: 2021-04-20 07:00:00 - Duration: 21.481027364730835,  I: 112, N: 1080199, D: 0, H: 0, P: 5000
8: 2021-04-20 08:00:00 - Duration: 35.609898805618286,  I: 117, N: 1080194, D: 0, H: 0, P: 5000
9: 2021-04-20 09:00:00 - Duration: 40.59369659423828,  I: 121, N: 1080190, D: 0, H: 0, P: 5000
10: 2021-04-20 10:00:00 - Duration: 37.820498

85: 2021-04-23 13:00:00 - Duration: 34.87980389595032,  I: 655, N: 1079649, D: 7, H: 0, P: 4923
86: 2021-04-23 14:00:00 - Duration: 36.04356598854065,  I: 657, N: 1079647, D: 7, H: 0, P: 4920
87: 2021-04-23 15:00:00 - Duration: 21.421384811401367,  I: 657, N: 1079647, D: 7, H: 0, P: 4917
88: 2021-04-23 16:00:00 - Duration: 22.030476570129395,  I: 659, N: 1079645, D: 7, H: 0, P: 4914
89: 2021-04-23 17:00:00 - Duration: 22.07146644592285,  I: 660, N: 1079643, D: 8, H: 0, P: 4913
90: 2021-04-23 18:00:00 - Duration: 22.90205979347229,  I: 660, N: 1079643, D: 8, H: 0, P: 4911
91: 2021-04-23 19:00:00 - Duration: 20.799002170562744,  I: 664, N: 1079639, D: 8, H: 0, P: 4910
92: 2021-04-23 20:00:00 - Duration: 20.49188494682312,  I: 666, N: 1079635, D: 10, H: 0, P: 4911
93: 2021-04-23 21:00:00 - Duration: 20.695363521575928,  I: 665, N: 1079634, D: 12, H: 0, P: 4913
94: 2021-04-23 22:00:00 - Duration: 21.376829385757446,  I: 666, N: 1079632, D: 13, H: 0, P: 4912
95: 2021-04-23 23:00:00 - Durati

169: 2021-04-27 01:00:00 - Duration: 21.550511837005615,  I: 930, N: 1079322, D: 59, H: 0, P: 4885
170: 2021-04-27 02:00:00 - Duration: 20.483503103256226,  I: 927, N: 1079322, D: 62, H: 0, P: 4886
171: 2021-04-27 03:00:00 - Duration: 22.326380968093872,  I: 928, N: 1079321, D: 62, H: 0, P: 4885
172: 2021-04-27 04:00:00 - Duration: 22.158978700637817,  I: 928, N: 1079321, D: 62, H: 0, P: 4885
173: 2021-04-27 05:00:00 - Duration: 21.53523588180542,  I: 929, N: 1079319, D: 63, H: 0, P: 4884
174: 2021-04-27 06:00:00 - Duration: 21.944921493530273,  I: 927, N: 1079319, D: 65, H: 0, P: 4885
175: 2021-04-27 07:00:00 - Duration: 21.918031692504883,  I: 926, N: 1079319, D: 66, H: 0, P: 4885
176: 2021-04-27 08:00:00 - Duration: 35.32506084442139,  I: 927, N: 1079318, D: 66, H: 0, P: 4884
177: 2021-04-27 09:00:00 - Duration: 40.02472543716431,  I: 926, N: 1079318, D: 67, H: 0, P: 4885
178: 2021-04-27 10:00:00 - Duration: 36.63056755065918,  I: 926, N: 1079318, D: 67, H: 0, P: 4882
179: 2021-04-2

252: 2021-04-30 12:00:00 - Duration: 35.92239475250244,  I: 901, N: 1079290, D: 120, H: 0, P: 4916
253: 2021-04-30 13:00:00 - Duration: 34.471168994903564,  I: 901, N: 1079290, D: 120, H: 0, P: 4916
254: 2021-04-30 14:00:00 - Duration: 35.75199604034424,  I: 900, N: 1079289, D: 122, H: 0, P: 4918
255: 2021-04-30 15:00:00 - Duration: 21.360930681228638,  I: 901, N: 1079287, D: 123, H: 0, P: 4919
256: 2021-04-30 16:00:00 - Duration: 21.614615440368652,  I: 902, N: 1079286, D: 123, H: 0, P: 4919
257: 2021-04-30 17:00:00 - Duration: 21.841546773910522,  I: 901, N: 1079286, D: 124, H: 0, P: 4920
258: 2021-04-30 18:00:00 - Duration: 22.884900331497192,  I: 901, N: 1079286, D: 124, H: 0, P: 4920
259: 2021-04-30 19:00:00 - Duration: 20.541069984436035,  I: 902, N: 1079285, D: 124, H: 0, P: 4920
260: 2021-04-30 20:00:00 - Duration: 20.942761421203613,  I: 901, N: 1079285, D: 125, H: 0, P: 4921
261: 2021-04-30 21:00:00 - Duration: 20.6989643573761,  I: 900, N: 1079285, D: 126, H: 0, P: 4922
262:

335: 2021-05-03 23:00:00 - Duration: 21.851622819900513,  I: 917, N: 1079244, D: 150, H: 0, P: 4939
336: 2021-05-04 00:00:00 - Duration: 21.32631015777588,  I: 917, N: 1079244, D: 150, H: 0, P: 4939
337: 2021-05-04 01:00:00 - Duration: 21.664923667907715,  I: 916, N: 1079244, D: 151, H: 0, P: 4940
338: 2021-05-04 02:00:00 - Duration: 21.556301832199097,  I: 835, N: 1079243, D: 151, H: 82, P: 4942
339: 2021-05-04 03:00:00 - Duration: 22.243731260299683,  I: 833, N: 1079242, D: 152, H: 84, P: 4943
340: 2021-05-04 04:00:00 - Duration: 22.367072582244873,  I: 833, N: 1079242, D: 152, H: 84, P: 4943
341: 2021-05-04 05:00:00 - Duration: 22.15082597732544,  I: 832, N: 1079242, D: 152, H: 85, P: 4943
342: 2021-05-04 06:00:00 - Duration: 21.66116213798523,  I: 831, N: 1079242, D: 152, H: 86, P: 4943
343: 2021-05-04 07:00:00 - Duration: 22.130999088287354,  I: 829, N: 1079242, D: 152, H: 88, P: 4943
344: 2021-05-04 08:00:00 - Duration: 36.21692442893982,  I: 828, N: 1079242, D: 152, H: 89, P: 49

416: 2021-05-07 08:00:00 - Duration: 36.32369875907898,  I: 407, N: 1079220, D: 166, H: 518, P: 4970
417: 2021-05-07 09:00:00 - Duration: 39.79523038864136,  I: 403, N: 1079220, D: 166, H: 522, P: 4970
418: 2021-05-07 10:00:00 - Duration: 36.81078505516052,  I: 395, N: 1079220, D: 166, H: 530, P: 4970
419: 2021-05-07 11:00:00 - Duration: 36.2173056602478,  I: 390, N: 1079220, D: 167, H: 534, P: 4971
420: 2021-05-07 12:00:00 - Duration: 35.16021680831909,  I: 385, N: 1079220, D: 169, H: 537, P: 4972
421: 2021-05-07 13:00:00 - Duration: 35.902790784835815,  I: 385, N: 1079220, D: 169, H: 537, P: 4972
422: 2021-05-07 14:00:00 - Duration: 34.970741748809814,  I: 384, N: 1079220, D: 169, H: 538, P: 4972
423: 2021-05-07 15:00:00 - Duration: 22.24647307395935,  I: 381, N: 1079220, D: 170, H: 540, P: 4973
424: 2021-05-07 16:00:00 - Duration: 21.515250205993652,  I: 379, N: 1079220, D: 170, H: 542, P: 4973
425: 2021-05-07 17:00:00 - Duration: 21.137126445770264,  I: 379, N: 1079220, D: 170, H: 

497: 2021-05-10 17:00:00 - Duration: 21.80960202217102,  I: 111, N: 1079201, D: 178, H: 821, P: 4991
498: 2021-05-10 18:00:00 - Duration: 23.68619394302368,  I: 111, N: 1079201, D: 178, H: 821, P: 4991
499: 2021-05-10 19:00:00 - Duration: 20.308362007141113,  I: 111, N: 1079201, D: 178, H: 821, P: 4991
500: 2021-05-10 20:00:00 - Duration: 21.44925045967102,  I: 110, N: 1079201, D: 178, H: 822, P: 4991
501: 2021-05-10 21:00:00 - Duration: 21.605337142944336,  I: 111, N: 1079200, D: 178, H: 822, P: 4991
502: 2021-05-10 22:00:00 - Duration: 20.926902532577515,  I: 111, N: 1079200, D: 178, H: 822, P: 4991
503: 2021-05-10 23:00:00 - Duration: 21.77576208114624,  I: 111, N: 1079200, D: 178, H: 822, P: 4991
504: 2021-05-11 00:00:00 - Duration: 22.24766492843628,  I: 111, N: 1079200, D: 178, H: 822, P: 4991
505: 2021-05-11 01:00:00 - Duration: 21.424819946289062,  I: 111, N: 1079200, D: 178, H: 822, P: 4991
506: 2021-05-11 02:00:00 - Duration: 21.44395422935486,  I: 110, N: 1079200, D: 178, H:

579: 2021-05-14 03:00:00 - Duration: 22.249566793441772,  I: 92, N: 1079191, D: 183, H: 845, P: 4995
580: 2021-05-14 04:00:00 - Duration: 22.059624671936035,  I: 92, N: 1079191, D: 183, H: 845, P: 4995
581: 2021-05-14 05:00:00 - Duration: 21.512030124664307,  I: 92, N: 1079191, D: 183, H: 845, P: 4995
582: 2021-05-14 06:00:00 - Duration: 21.925498723983765,  I: 92, N: 1079191, D: 183, H: 845, P: 4995
583: 2021-05-14 07:00:00 - Duration: 21.932051420211792,  I: 92, N: 1079191, D: 183, H: 845, P: 4995
584: 2021-05-14 08:00:00 - Duration: 36.137211084365845,  I: 89, N: 1079190, D: 183, H: 849, P: 4995
585: 2021-05-14 09:00:00 - Duration: 39.37205219268799,  I: 89, N: 1079190, D: 183, H: 849, P: 4995
586: 2021-05-14 10:00:00 - Duration: 36.64991235733032,  I: 89, N: 1079190, D: 183, H: 849, P: 4995
587: 2021-05-14 11:00:00 - Duration: 35.401313066482544,  I: 89, N: 1079190, D: 183, H: 849, P: 4995
588: 2021-05-14 12:00:00 - Duration: 35.71981120109558,  I: 89, N: 1079190, D: 183, H: 849, P

661: 2021-05-17 13:00:00 - Duration: 35.149105072021484,  I: 61, N: 1079180, D: 185, H: 885, P: 4995
662: 2021-05-17 14:00:00 - Duration: 35.45444393157959,  I: 61, N: 1079180, D: 185, H: 885, P: 4995
663: 2021-05-17 15:00:00 - Duration: 21.43423342704773,  I: 61, N: 1079180, D: 185, H: 885, P: 4995
664: 2021-05-17 16:00:00 - Duration: 21.12716293334961,  I: 61, N: 1079180, D: 185, H: 885, P: 4995
665: 2021-05-17 17:00:00 - Duration: 21.440191745758057,  I: 60, N: 1079180, D: 185, H: 886, P: 4995
666: 2021-05-17 18:00:00 - Duration: 23.460611581802368,  I: 59, N: 1079180, D: 185, H: 887, P: 4995
667: 2021-05-17 19:00:00 - Duration: 20.092087745666504,  I: 60, N: 1079179, D: 185, H: 887, P: 4995
668: 2021-05-17 20:00:00 - Duration: 20.782310962677002,  I: 59, N: 1079179, D: 185, H: 888, P: 4995
669: 2021-05-17 21:00:00 - Duration: 21.34476923942566,  I: 59, N: 1079179, D: 185, H: 888, P: 4995
670: 2021-05-17 22:00:00 - Duration: 20.686089277267456,  I: 59, N: 1079179, D: 185, H: 888, P:

743: 2021-05-20 23:00:00 - Duration: 21.372695207595825,  I: 47, N: 1079174, D: 185, H: 905, P: 4996
744: 2021-05-21 00:00:00 - Duration: 21.420618057250977,  I: 47, N: 1079174, D: 185, H: 905, P: 4996


In [ ]:
print(path.join('C:\\', 'Users', 'Suro', 'Desktop', 'Capstone', 'evid', checkpoint_directory,
                            f'checkpoint_{model.date}.pickle'))


In [ ]:
from pathlib import Path, PureWindowsPath

In [ ]:
file_name = Path(f'{checkpoint_directory}/checkpoint_{model.date}.pickle')
path_windows = PureWindowsPath(file_name)
print(path_windows)

In [ ]:
        with open(str(path.join(checkpoint_directory,f'checkpoint_{model.date}.pickle'.replace(":", "_"))), 'wb') as f:
            pass

In [ ]:
print(str(path.join(checkpoint_directory,f'checkpoint_{model.date}.pickle')))

In [37]:
path.exists(str(path.join('experiments','1','checkpoint_2021-05-05 00_00_00.pickle')))

True

In [9]:
import pickle


In [16]:
with open(str(path.join('experiments','all_maks','checkpoint_2021-05-05 00_00_00.pickle')), 'rb') as f:
    model = pickle.load(f)

In [17]:
from configs.move_distribution_tensor_lockdown import fillTensorExperimentLockDown
move_distribution_tensor = np.zeros(shape=(7, 5, 24, 10))
fillTensorExperimentLockDown(move_distribution_tensor)

In [18]:
model.moving_distribution_tensor= move_distribution_tensor
print(model.moving_distribution_tensor[1,3,20])

[0.  0.  0.  0.  0.  0.1 0.  0.  0.  0.9]


In [19]:
experiment_name = '7'
config_path = path.join('configs','conf_all_maks.json')
with open(config_path) as json_file:
    configs = json.load(json_file)
model.checkpoint_directory = path.join('experiments', experiment_name)
model.inf_prob_args = configs['compute_inf_prob_arguments']
print(model.checkpoint_directory)
print(model.inf_prob_args)

experiments\7
{'school': {'vol': 2, 'rlwr': 2, 'area': 2500, 'height': 3, 'mask_out': 0.3, 'mask_in': 0.6}, 'university': {'vol': 2, 'rlwr': 2, 'area': 7000, 'height': 3, 'mask_out': 0.3, 'mask_in': 0.6}, 'kindergarten': {'vol': 2, 'rlwr': 2, 'area': 1000, 'height': 3, 'mask_out': 0.3, 'mask_in': 0.6}, 'work': {'vol': 2, 'area': 1200, 'height': 3, 'rlwr': 2, 'mask_out': 0.3, 'mask_in': 0.6}, 'cafe': {'vol': 2, 'area': 450, 'height': 4, 'rlwr': 3, 'mask_out': 0, 'mask_in': 0}, 'church': {'vol': 1, 'area': 300, 'height': 15, 'rlwr': 2, 'mask_out': 0.3, 'mask_in': 0.6}, 'sport': {'vol': 2, 'area': 1000, 'height': 4, 'rlwr': 2, 'mask_out': 0, 'mask_in': 0}, 'shop': {'vol': 3, 'area': 550, 'height': 5, 'rlwr': 3, 'mask_out': 0.3, 'mask_in': 0.6}, 'building': {'vol': 2, 'area': 70, 'height': 3, 'rlwr': 3, 'mask_out': 0, 'mask_in': 0}, 'hospital': {'vol': 2, 'area': 30, 'height': 3, 'rlwr': 3, 'mask_out': 0.8, 'mask_in': 0.8}}


In [16]:
move_distribution_tensor[1,3,20]

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.2  , 0.   , 0.   , 0.102,
       0.698])

In [22]:
print(model.date)
print(model.moving_distribution_tensor[1,3,18])

2021-05-05 00:00:00
[0.      0.00125 0.      0.      0.      0.15    0.      0.      0.70375
 0.145  ]


In [20]:
n_step = 744
for i in range(361, n_step+1):
    date = model.date
    t = time.time()
    model.step()
    d = time.time() - t
    r = model.datacollector.get_model_vars_dataframe().iloc[-1].values
    print(f'{i}: {date} - Duration: {d},  I: {r[0]}, N: {r[1]}, D: {r[2]}, H: {r[3]}, P: {r[4]}')

361: 2021-05-05 00:00:00 - Duration: 28.705031394958496,  I: 781, N: 1079235, D: 156, H: 139, P: 4947
362: 2021-05-05 01:00:00 - Duration: 27.850693941116333,  I: 770, N: 1079234, D: 156, H: 151, P: 4947
363: 2021-05-05 02:00:00 - Duration: 28.42992115020752,  I: 763, N: 1079234, D: 156, H: 158, P: 4947
364: 2021-05-05 03:00:00 - Duration: 31.012474060058594,  I: 744, N: 1079234, D: 157, H: 176, P: 4949
365: 2021-05-05 04:00:00 - Duration: 30.07393193244934,  I: 733, N: 1079232, D: 157, H: 189, P: 4949
366: 2021-05-05 05:00:00 - Duration: 29.972527265548706,  I: 715, N: 1079232, D: 158, H: 206, P: 4950
367: 2021-05-05 06:00:00 - Duration: 30.885945558547974,  I: 681, N: 1079232, D: 158, H: 240, P: 4956
368: 2021-05-05 07:00:00 - Duration: 32.206183195114136,  I: 652, N: 1079232, D: 158, H: 269, P: 4959
369: 2021-05-05 08:00:00 - Duration: 28.286126136779785,  I: 625, N: 1079232, D: 158, H: 296, P: 4959
370: 2021-05-05 09:00:00 - Duration: 28.808693885803223,  I: 608, N: 1079231, D: 158

442: 2021-05-08 09:00:00 - Duration: 32.85258722305298,  I: 295, N: 1079213, D: 165, H: 638, P: 4973
443: 2021-05-08 10:00:00 - Duration: 30.853570461273193,  I: 283, N: 1079213, D: 165, H: 650, P: 4974
444: 2021-05-08 11:00:00 - Duration: 29.99725914001465,  I: 270, N: 1079212, D: 165, H: 664, P: 4975
445: 2021-05-08 12:00:00 - Duration: 29.449360370635986,  I: 256, N: 1079212, D: 165, H: 678, P: 4975
446: 2021-05-08 13:00:00 - Duration: 27.54318642616272,  I: 248, N: 1079212, D: 165, H: 686, P: 4975
447: 2021-05-08 14:00:00 - Duration: 26.9765145778656,  I: 241, N: 1079212, D: 165, H: 693, P: 4976
448: 2021-05-08 15:00:00 - Duration: 26.906029224395752,  I: 236, N: 1079212, D: 165, H: 698, P: 4977
449: 2021-05-08 16:00:00 - Duration: 27.53462505340576,  I: 231, N: 1079212, D: 165, H: 703, P: 4977
450: 2021-05-08 17:00:00 - Duration: 30.620251178741455,  I: 225, N: 1079212, D: 165, H: 709, P: 4978
451: 2021-05-08 18:00:00 - Duration: 27.194488286972046,  I: 220, N: 1079211, D: 165, H:

523: 2021-05-11 18:00:00 - Duration: 27.10285472869873,  I: 109, N: 1079198, D: 172, H: 832, P: 4987
524: 2021-05-11 19:00:00 - Duration: 27.25504446029663,  I: 109, N: 1079198, D: 172, H: 832, P: 4987
525: 2021-05-11 20:00:00 - Duration: 27.360082387924194,  I: 110, N: 1079197, D: 172, H: 832, P: 4987
526: 2021-05-11 21:00:00 - Duration: 25.87827968597412,  I: 111, N: 1079196, D: 172, H: 832, P: 4987
527: 2021-05-11 22:00:00 - Duration: 26.229301929473877,  I: 111, N: 1079196, D: 172, H: 832, P: 4987
528: 2021-05-11 23:00:00 - Duration: 27.180035829544067,  I: 111, N: 1079196, D: 172, H: 832, P: 4987
529: 2021-05-12 00:00:00 - Duration: 26.461184978485107,  I: 112, N: 1079195, D: 172, H: 832, P: 4986
530: 2021-05-12 01:00:00 - Duration: 26.731048345565796,  I: 112, N: 1079194, D: 172, H: 833, P: 4986
531: 2021-05-12 02:00:00 - Duration: 25.665393829345703,  I: 112, N: 1079194, D: 172, H: 833, P: 4986
532: 2021-05-12 03:00:00 - Duration: 25.43866515159607,  I: 111, N: 1079194, D: 172, 

605: 2021-05-15 04:00:00 - Duration: 27.680404663085938,  I: 87, N: 1079188, D: 173, H: 863, P: 4987
606: 2021-05-15 05:00:00 - Duration: 29.417267322540283,  I: 85, N: 1079188, D: 174, H: 864, P: 4988
607: 2021-05-15 06:00:00 - Duration: 28.05301332473755,  I: 85, N: 1079188, D: 174, H: 864, P: 4988
608: 2021-05-15 07:00:00 - Duration: 29.71696925163269,  I: 82, N: 1079188, D: 174, H: 867, P: 4988
609: 2021-05-15 08:00:00 - Duration: 28.83344602584839,  I: 81, N: 1079188, D: 174, H: 868, P: 4988
610: 2021-05-15 09:00:00 - Duration: 28.78057312965393,  I: 80, N: 1079188, D: 174, H: 869, P: 4988
611: 2021-05-15 10:00:00 - Duration: 28.790498971939087,  I: 79, N: 1079188, D: 174, H: 870, P: 4988
612: 2021-05-15 11:00:00 - Duration: 28.904718160629272,  I: 80, N: 1079186, D: 174, H: 871, P: 4988
613: 2021-05-15 12:00:00 - Duration: 29.50964093208313,  I: 80, N: 1079186, D: 174, H: 871, P: 4988
614: 2021-05-15 13:00:00 - Duration: 29.824819326400757,  I: 80, N: 1079186, D: 174, H: 871, P: 

687: 2021-05-18 14:00:00 - Duration: 25.925832271575928,  I: 54, N: 1079178, D: 176, H: 903, P: 4990
688: 2021-05-18 15:00:00 - Duration: 27.080403804779053,  I: 54, N: 1079178, D: 176, H: 903, P: 4990
689: 2021-05-18 16:00:00 - Duration: 26.422970056533813,  I: 54, N: 1079178, D: 176, H: 903, P: 4990
690: 2021-05-18 17:00:00 - Duration: 27.26042938232422,  I: 52, N: 1079178, D: 176, H: 905, P: 4989
691: 2021-05-18 18:00:00 - Duration: 26.19073247909546,  I: 52, N: 1079178, D: 176, H: 905, P: 4989
692: 2021-05-18 19:00:00 - Duration: 26.782829761505127,  I: 52, N: 1079178, D: 176, H: 905, P: 4989
693: 2021-05-18 20:00:00 - Duration: 27.24124813079834,  I: 52, N: 1079178, D: 176, H: 905, P: 4989
694: 2021-05-18 21:00:00 - Duration: 26.456190824508667,  I: 52, N: 1079178, D: 176, H: 905, P: 4989
695: 2021-05-18 22:00:00 - Duration: 26.892465114593506,  I: 50, N: 1079178, D: 177, H: 906, P: 4990
696: 2021-05-18 23:00:00 - Duration: 26.784334421157837,  I: 50, N: 1079178, D: 177, H: 906, P

# Experiment 4

In [21]:
with open(str(path.join('experiments','1','checkpoint_2021-05-05 00_00_00.pickle')), 'rb') as f:
    model = pickle.load(f)

In [22]:
from configs.move_distribution_experiment_4 import fillTensorExperiment4
move_distribution_tensor = np.zeros(shape=(7, 5, 24, 10))
fillTensorExperiment4(move_distribution_tensor)
model.moving_distribution_tensor= move_distribution_tensor
print(model.moving_distribution_tensor[1,3,20])

[0.    0.    0.    0.    0.    0.1   0.    0.    0.102 0.798]


In [23]:
experiment_name = '4'
config_path = path.join('configs','conf_1.json')
with open(config_path) as json_file:
    configs = json.load(json_file)
model.checkpoint_directory = path.join('experiments', experiment_name)
model.inf_prob_args = configs['compute_inf_prob_arguments']
print(model.checkpoint_directory)
print(model.inf_prob_args)

experiments\4
{'school': {'vol': 2, 'rlwr': 2, 'area': 2500, 'height': 3, 'mask_out': 0, 'mask_in': 0}, 'university': {'vol': 2, 'rlwr': 2, 'area': 7000, 'height': 3, 'mask_out': 0, 'mask_in': 0}, 'kindergarten': {'vol': 2, 'rlwr': 2, 'area': 1000, 'height': 3, 'mask_out': 0, 'mask_in': 0}, 'work': {'vol': 2, 'area': 1200, 'height': 3, 'rlwr': 2, 'mask_out': 0, 'mask_in': 0}, 'cafe': {'vol': 2, 'area': 450, 'height': 4, 'rlwr': 3, 'mask_out': 0, 'mask_in': 0}, 'church': {'vol': 1, 'area': 300, 'height': 15, 'rlwr': 2, 'mask_out': 0, 'mask_in': 0}, 'sport': {'vol': 2, 'area': 1000, 'height': 4, 'rlwr': 2, 'mask_out': 0, 'mask_in': 0}, 'shop': {'vol': 3, 'area': 550, 'height': 5, 'rlwr': 3, 'mask_out': 0, 'mask_in': 0}, 'building': {'vol': 2, 'area': 70, 'height': 3, 'rlwr': 3, 'mask_out': 0, 'mask_in': 0}, 'hospital': {'vol': 2, 'area': 30, 'height': 3, 'rlwr': 3, 'mask_out': 0.8, 'mask_in': 0.8}}


In [24]:
n_step = 744
for i in range(361, n_step+1):
    date = model.date
    t = time.time()
    model.step()
    d = time.time() - t
    r = model.datacollector.get_model_vars_dataframe().iloc[-1].values
    print(f'{i}: {date} - Duration: {d},  I: {r[0]}, N: {r[1]}, D: {r[2]}, H: {r[3]}, P: {r[4]}')

361: 2021-05-05 00:00:00 - Duration: 32.18206810951233,  I: 2072, N: 1077875, D: 240, H: 124, P: 4789
362: 2021-05-05 01:00:00 - Duration: 31.283651113510132,  I: 2071, N: 1077873, D: 240, H: 127, P: 4789
363: 2021-05-05 02:00:00 - Duration: 31.118852376937866,  I: 2065, N: 1077872, D: 243, H: 131, P: 4790
364: 2021-05-05 03:00:00 - Duration: 32.803606271743774,  I: 2069, N: 1077866, D: 243, H: 133, P: 4790
365: 2021-05-05 04:00:00 - Duration: 32.69380784034729,  I: 2072, N: 1077857, D: 244, H: 138, P: 4791
366: 2021-05-05 05:00:00 - Duration: 32.09692406654358,  I: 2068, N: 1077844, D: 246, H: 153, P: 4793
367: 2021-05-05 06:00:00 - Duration: 32.300785064697266,  I: 2059, N: 1077836, D: 247, H: 169, P: 4797
368: 2021-05-05 07:00:00 - Duration: 32.235729455947876,  I: 2054, N: 1077823, D: 247, H: 187, P: 4794
369: 2021-05-05 08:00:00 - Duration: 34.50157976150513,  I: 2055, N: 1077804, D: 248, H: 204, P: 4795
370: 2021-05-05 09:00:00 - Duration: 38.80324101448059,  I: 2050, N: 1077786,

442: 2021-05-08 09:00:00 - Duration: 34.92947268486023,  I: 2449, N: 1076799, D: 312, H: 751, P: 4752
443: 2021-05-08 10:00:00 - Duration: 34.96646475791931,  I: 2448, N: 1076783, D: 313, H: 767, P: 4752
444: 2021-05-08 11:00:00 - Duration: 35.42597937583923,  I: 2444, N: 1076772, D: 315, H: 780, P: 4753
445: 2021-05-08 12:00:00 - Duration: 35.8054301738739,  I: 2443, N: 1076761, D: 315, H: 792, P: 4747
446: 2021-05-08 13:00:00 - Duration: 35.34973859786987,  I: 2444, N: 1076751, D: 316, H: 800, P: 4744
447: 2021-05-08 14:00:00 - Duration: 35.206056118011475,  I: 2439, N: 1076742, D: 318, H: 812, P: 4739
448: 2021-05-08 15:00:00 - Duration: 35.098156690597534,  I: 2439, N: 1076731, D: 319, H: 822, P: 4736
449: 2021-05-08 16:00:00 - Duration: 34.69163227081299,  I: 2438, N: 1076725, D: 320, H: 828, P: 4735
450: 2021-05-08 17:00:00 - Duration: 34.60098338127136,  I: 2437, N: 1076715, D: 320, H: 839, P: 4732
451: 2021-05-08 18:00:00 - Duration: 34.20965075492859,  I: 2438, N: 1076706, D: 

522: 2021-05-11 17:00:00 - Duration: 31.35078454017639,  I: 2386, N: 1076250, D: 420, H: 1255, P: 4669
523: 2021-05-11 18:00:00 - Duration: 33.14957046508789,  I: 2381, N: 1076249, D: 423, H: 1258, P: 4669
524: 2021-05-11 19:00:00 - Duration: 31.73703932762146,  I: 2378, N: 1076246, D: 425, H: 1262, P: 4671
525: 2021-05-11 20:00:00 - Duration: 31.982759952545166,  I: 2377, N: 1076241, D: 428, H: 1265, P: 4674
526: 2021-05-11 21:00:00 - Duration: 32.30268740653992,  I: 2376, N: 1076238, D: 429, H: 1268, P: 4674
527: 2021-05-11 22:00:00 - Duration: 32.70441913604736,  I: 2381, N: 1076230, D: 431, H: 1269, P: 4676
528: 2021-05-11 23:00:00 - Duration: 32.723987340927124,  I: 2386, N: 1076222, D: 432, H: 1271, P: 4676
529: 2021-05-12 00:00:00 - Duration: 32.97223138809204,  I: 2387, N: 1076218, D: 432, H: 1274, P: 4677
530: 2021-05-12 01:00:00 - Duration: 32.878095388412476,  I: 2392, N: 1076208, D: 435, H: 1276, P: 4680
531: 2021-05-12 02:00:00 - Duration: 32.766879081726074,  I: 2394, N: 

602: 2021-05-15 01:00:00 - Duration: 32.53696584701538,  I: 2587, N: 1075649, D: 549, H: 1526, P: 4742
603: 2021-05-15 02:00:00 - Duration: 32.98210000991821,  I: 2602, N: 1075626, D: 554, H: 1529, P: 4746
604: 2021-05-15 03:00:00 - Duration: 33.83013463020325,  I: 2615, N: 1075609, D: 555, H: 1532, P: 4747
605: 2021-05-15 04:00:00 - Duration: 34.01809024810791,  I: 2643, N: 1075572, D: 557, H: 1539, P: 4749
606: 2021-05-15 05:00:00 - Duration: 33.87737274169922,  I: 2668, N: 1075539, D: 558, H: 1546, P: 4750
607: 2021-05-15 06:00:00 - Duration: 34.048250675201416,  I: 2672, N: 1075517, D: 560, H: 1562, P: 4754
608: 2021-05-15 07:00:00 - Duration: 34.32991552352905,  I: 2690, N: 1075490, D: 560, H: 1571, P: 4755
609: 2021-05-15 08:00:00 - Duration: 34.7770791053772,  I: 2696, N: 1075471, D: 562, H: 1582, P: 4757
610: 2021-05-15 09:00:00 - Duration: 34.96699547767639,  I: 2707, N: 1075448, D: 562, H: 1594, P: 4756
611: 2021-05-15 10:00:00 - Duration: 34.89848303794861,  I: 2702, N: 1075

682: 2021-05-18 09:00:00 - Duration: 37.42661213874817,  I: 2676, N: 1074969, D: 674, H: 1992, P: 4728
683: 2021-05-18 10:00:00 - Duration: 34.876070499420166,  I: 2667, N: 1074969, D: 675, H: 2000, P: 4728
684: 2021-05-18 11:00:00 - Duration: 33.887383460998535,  I: 2668, N: 1074964, D: 677, H: 2002, P: 4728
685: 2021-05-18 12:00:00 - Duration: 32.976279973983765,  I: 2670, N: 1074958, D: 677, H: 2006, P: 4725
686: 2021-05-18 13:00:00 - Duration: 32.54665970802307,  I: 2668, N: 1074956, D: 679, H: 2008, P: 4727
687: 2021-05-18 14:00:00 - Duration: 31.971251964569092,  I: 2666, N: 1074954, D: 682, H: 2009, P: 4730
688: 2021-05-18 15:00:00 - Duration: 30.996710777282715,  I: 2673, N: 1074945, D: 683, H: 2010, P: 4730
689: 2021-05-18 16:00:00 - Duration: 30.584742784500122,  I: 2672, N: 1074938, D: 685, H: 2016, P: 4732
690: 2021-05-18 17:00:00 - Duration: 30.74177861213684,  I: 2672, N: 1074937, D: 685, H: 2017, P: 4730
691: 2021-05-18 18:00:00 - Duration: 32.56176686286926,  I: 2675, N

# Experiment 9

In [10]:
with open(str(path.join('experiments','all_maks','checkpoint_2021-05-05 00_00_00.pickle')), 
          'rb') as f:
    model = pickle.load(f)

In [11]:
from configs.move_distribution_tensor_lockdown import fillTensorExperimentLockDown
move_distribution_tensor = np.zeros(shape=(7, 5, 24, 10))
fillTensorExperimentLockDown(move_distribution_tensor)
model.moving_distribution_tensor= move_distribution_tensor
print(model.moving_distribution_tensor[1,3,20])

[0.  0.  0.  0.  0.  0.1 0.  0.  0.  0.9]


In [12]:
experiment_name = '9'
config_path = path.join('configs','conf_1.json')
with open(config_path) as json_file:
    configs = json.load(json_file)
model.checkpoint_directory = path.join('experiments', experiment_name)
model.inf_prob_args = configs['compute_inf_prob_arguments']
print(model.checkpoint_directory)
print(model.inf_prob_args)

experiments\9
{'school': {'vol': 2, 'rlwr': 2, 'area': 2500, 'height': 3, 'mask_out': 0, 'mask_in': 0}, 'university': {'vol': 2, 'rlwr': 2, 'area': 7000, 'height': 3, 'mask_out': 0, 'mask_in': 0}, 'kindergarten': {'vol': 2, 'rlwr': 2, 'area': 1000, 'height': 3, 'mask_out': 0, 'mask_in': 0}, 'work': {'vol': 2, 'area': 1200, 'height': 3, 'rlwr': 2, 'mask_out': 0, 'mask_in': 0}, 'cafe': {'vol': 2, 'area': 450, 'height': 4, 'rlwr': 3, 'mask_out': 0, 'mask_in': 0}, 'church': {'vol': 1, 'area': 300, 'height': 15, 'rlwr': 2, 'mask_out': 0, 'mask_in': 0}, 'sport': {'vol': 2, 'area': 1000, 'height': 4, 'rlwr': 2, 'mask_out': 0, 'mask_in': 0}, 'shop': {'vol': 3, 'area': 550, 'height': 5, 'rlwr': 3, 'mask_out': 0, 'mask_in': 0}, 'building': {'vol': 2, 'area': 70, 'height': 3, 'rlwr': 3, 'mask_out': 0, 'mask_in': 0}, 'hospital': {'vol': 2, 'area': 30, 'height': 3, 'rlwr': 3, 'mask_out': 0.8, 'mask_in': 0.8}}


In [ ]:
n_step = 744
for i in range(361, n_step+1):
    date = model.date
    t = time.time()
    model.step()
    d = time.time() - t
    r = model.datacollector.get_model_vars_dataframe().iloc[-1].values
    print(f'{i}: {date} - Duration: {d},  I: {r[0]}, N: {r[1]}, D: {r[2]}, H: {r[3]}, P: {r[4]}')

361: 2021-05-05 00:00:00 - Duration: 23.214673042297363,  I: 781, N: 1079235, D: 156, H: 139, P: 4947
362: 2021-05-05 01:00:00 - Duration: 23.131511449813843,  I: 770, N: 1079234, D: 156, H: 151, P: 4947
363: 2021-05-05 02:00:00 - Duration: 23.786548614501953,  I: 763, N: 1079234, D: 156, H: 158, P: 4947
364: 2021-05-05 03:00:00 - Duration: 25.429861545562744,  I: 747, N: 1079232, D: 156, H: 176, P: 4948
365: 2021-05-05 04:00:00 - Duration: 24.560952186584473,  I: 734, N: 1079232, D: 156, H: 189, P: 4948
366: 2021-05-05 05:00:00 - Duration: 24.796417474746704,  I: 717, N: 1079232, D: 156, H: 206, P: 4948
367: 2021-05-05 06:00:00 - Duration: 24.23716902732849,  I: 683, N: 1079232, D: 156, H: 240, P: 4954
368: 2021-05-05 07:00:00 - Duration: 24.6868793964386,  I: 655, N: 1079231, D: 156, H: 269, P: 4957
369: 2021-05-05 08:00:00 - Duration: 24.405394554138184,  I: 628, N: 1079231, D: 156, H: 296, P: 4957
370: 2021-05-05 09:00:00 - Duration: 24.1048903465271,  I: 611, N: 1079230, D: 156, H

442: 2021-05-08 09:00:00 - Duration: 35.3522891998291,  I: 294, N: 1079207, D: 175, H: 635, P: 4980
443: 2021-05-08 10:00:00 - Duration: 34.66322946548462,  I: 285, N: 1079206, D: 175, H: 645, P: 4980
444: 2021-05-08 11:00:00 - Duration: 34.852580308914185,  I: 271, N: 1079206, D: 175, H: 659, P: 4981
445: 2021-05-08 12:00:00 - Duration: 35.68203592300415,  I: 257, N: 1079206, D: 175, H: 673, P: 4981
446: 2021-05-08 13:00:00 - Duration: 33.88591432571411,  I: 249, N: 1079206, D: 175, H: 681, P: 4981
447: 2021-05-08 14:00:00 - Duration: 29.997912883758545,  I: 242, N: 1079206, D: 175, H: 688, P: 4982
448: 2021-05-08 15:00:00 - Duration: 32.4088659286499,  I: 237, N: 1079206, D: 175, H: 693, P: 4983
449: 2021-05-08 16:00:00 - Duration: 33.4769926071167,  I: 232, N: 1079206, D: 175, H: 698, P: 4983
450: 2021-05-08 17:00:00 - Duration: 30.215799808502197,  I: 226, N: 1079205, D: 176, H: 704, P: 4985
451: 2021-05-08 18:00:00 - Duration: 37.521005392074585,  I: 221, N: 1079205, D: 176, H: 70

523: 2021-05-11 18:00:00 - Duration: 32.75695180892944,  I: 110, N: 1079192, D: 182, H: 827, P: 4994
524: 2021-05-11 19:00:00 - Duration: 33.92576766014099,  I: 110, N: 1079192, D: 182, H: 827, P: 4994
525: 2021-05-11 20:00:00 - Duration: 36.7629508972168,  I: 110, N: 1079192, D: 182, H: 827, P: 4994
526: 2021-05-11 21:00:00 - Duration: 32.3797562122345,  I: 109, N: 1079192, D: 183, H: 827, P: 4995
527: 2021-05-11 22:00:00 - Duration: 31.15674352645874,  I: 109, N: 1079192, D: 183, H: 827, P: 4995
528: 2021-05-11 23:00:00 - Duration: 35.285329818725586,  I: 109, N: 1079192, D: 183, H: 827, P: 4995
529: 2021-05-12 00:00:00 - Duration: 30.240466356277466,  I: 111, N: 1079190, D: 183, H: 827, P: 4995
530: 2021-05-12 01:00:00 - Duration: 31.509770393371582,  I: 111, N: 1079189, D: 183, H: 828, P: 4995
531: 2021-05-12 02:00:00 - Duration: 30.136459827423096,  I: 112, N: 1079188, D: 183, H: 828, P: 4995
532: 2021-05-12 03:00:00 - Duration: 28.503185749053955,  I: 111, N: 1079188, D: 183, H: 

604: 2021-05-15 03:00:00 - Duration: 29.43511128425598,  I: 118, N: 1079152, D: 186, H: 855, P: 4997
605: 2021-05-15 04:00:00 - Duration: 37.07285976409912,  I: 126, N: 1079144, D: 186, H: 855, P: 4997
606: 2021-05-15 05:00:00 - Duration: 33.77699875831604,  I: 129, N: 1079140, D: 186, H: 856, P: 4997
607: 2021-05-15 06:00:00 - Duration: 30.880732536315918,  I: 134, N: 1079135, D: 186, H: 856, P: 4997
608: 2021-05-15 07:00:00 - Duration: 48.802942752838135,  I: 135, N: 1079131, D: 186, H: 859, P: 4997
609: 2021-05-15 08:00:00 - Duration: 37.658968925476074,  I: 144, N: 1079121, D: 186, H: 860, P: 4997
610: 2021-05-15 09:00:00 - Duration: 41.34298086166382,  I: 155, N: 1079109, D: 186, H: 861, P: 4997
611: 2021-05-15 10:00:00 - Duration: 38.175572633743286,  I: 164, N: 1079099, D: 186, H: 862, P: 4996
612: 2021-05-15 11:00:00 - Duration: 58.12736988067627,  I: 170, N: 1079092, D: 186, H: 863, P: 4996
613: 2021-05-15 12:00:00 - Duration: 44.74494934082031,  I: 181, N: 1079081, D: 186, H: